In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath = '/content/drive/MyDrive/Colab Notebooks/Copy of mLabel_tweets.csv'
df = pd.read_csv(filepath)


In [ ]:
import re
def preprocess_tweet(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions (@username or @ username)
    text = re.sub(r'\@\s?\w+', '', text)

    # Remove hash tags (#hashtag or # hashtag)
    text = re.sub(r'\#\s?\w+', '', text)

    # Convert text to lowercase (optional)
    text = text.lower()

    # Remove extra whitespaces
    text = text.strip()

    return text





In [ ]:
df['cleaned_tweet']=df['tweet'].apply(preprocess_tweet)

In [ ]:
df.head

<bound method NDFrame.head of         ID                                              tweet  \
0        1  @cath__kath AstraZeneca is made with the kidne...   
1        2  It begins. Please find safe alternatives to th...   
2        3  @PaolaQP1231 Well, I mean congratulations Covi...   
3        4  @BorisJohnson for those of us that do not wish...   
4        5  She has been trying to speak out: writing lett...   
...    ...                                                ...   
9916  9917  Former Pfizer Chief Scientific Officer on Expe...   
9917  9918  @garygilligan Not what the manufacturer's are ...   
9918  9919  //That's a complete no for now on the Oxford/A...   
9919  9920  Opinion: Vaccine side effects..- possible to h...   
9920  9921  @BorisJohnson resign now before you destroy th...   

                  labels                                      cleaned_tweet  
0            ingredients  astrazeneca is made with the kidney cells of a...  
1            side-effect  it begi

In [ ]:
df.columns

Index(['ID', 'tweet', 'labels', 'cleaned_tweet'], dtype='object')

In [ ]:
df.head

<bound method NDFrame.head of         ID                                              tweet  \
0        1  @cath__kath AstraZeneca is made with the kidne...   
1        2  It begins. Please find safe alternatives to th...   
2        3  @PaolaQP1231 Well, I mean congratulations Covi...   
3        4  @BorisJohnson for those of us that do not wish...   
4        5  She has been trying to speak out: writing lett...   
...    ...                                                ...   
9916  9917  Former Pfizer Chief Scientific Officer on Expe...   
9917  9918  @garygilligan Not what the manufacturer's are ...   
9918  9919  //That's a complete no for now on the Oxford/A...   
9919  9920  Opinion: Vaccine side effects..- possible to h...   
9920  9921  @BorisJohnson resign now before you destroy th...   

                  labels                                      cleaned_tweet  
0            ingredients  astrazeneca is made with the kidney cells of a...  
1            side-effect  it begi

In [ ]:
from collections import Counter

label_counts = Counter()

for label_string in df['labels']:
    individual_labels = label_string.split()
    label_counts.update(individual_labels)

# Sort labels based on counts in descending order
sorted_labels = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)

# Display the sorted counts
for label, count in sorted_labels:
    print(f"{label}: {count}")


side-effect: 3805
ineffective: 1672
rushed: 1477
pharma: 1273
mandatory: 783
unnecessary: 722
none: 629
political: 626
conspiracy: 487
ingredients: 436
country: 201
religious: 64


In [ ]:
distinct_labels = set()

for label_string in df['labels']:


    individual_labels = label_string.split()


    for label in individual_labels:
        distinct_labels.add(label)


distinct_labels_list = list(distinct_labels)
print(distinct_labels_list)
type(distinct_labels_list)
len(distinct_labels_list)

['ineffective', 'conspiracy', 'rushed', 'pharma', 'ingredients', 'country', 'religious', 'unnecessary', 'none', 'side-effect', 'political', 'mandatory']


12

In [ ]:
df['labels'].shape

(9921,)

#Sample with DistillBert

In [ ]:
pip install transformers torch

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
text = "DistilBERT is amazing!"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
inputs


{'input_ids': tensor([[  101,  4487, 16643, 23373,  2003,  6429,   999,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
tokens = tokenizer.tokenize(text)
print(tokens)

['di', '##sti', '##lbert', 'is', 'amazing', '!']


In [ ]:
input_ids_list = inputs['input_ids'][0].tolist()  # Convert tensor to Python list

for id in input_ids_list:
    print(f"ID: {id} -> Token: {tokenizer.decode(id)}")

ID: 101 -> Token: [ C L S ]
ID: 4487 -> Token: d i
ID: 16643 -> Token: # # s t i
ID: 23373 -> Token: # # l b e r t
ID: 2003 -> Token: i s
ID: 6429 -> Token: a m a z i n g
ID: 999 -> Token: !
ID: 102 -> Token: [ S E P ]


In [ ]:
with torch.no_grad():  # We don't need gradients for this demonstration
    outputs = model(**inputs)


###returns a tensor of shape [1, 7, 768] (assuming you're using DistilBERT-base and a batch size of 1).**

###The output tensor can be visualized as a 3D array:

###The first dimension (of size 1) is for each input sequence.
###The second dimension (of size 7) corresponds to each token.
###The third dimension (of size 768) is the embedding vector for each token.
###For our example, the output tensor is basically a list of 7 vectors, each of size 768. Each vector is a contextual embedding of the corresponding token.

In [ ]:
cls_embedding = outputs.last_hidden_state[:, 0, :]
cls_embedding1 = outputs.last_hidden_state[:, 1, :]
cls_embedding2 = outputs.last_hidden_state[:, 7, :]
cls_embedding3 = outputs.last_hidden_state[0]

In [ ]:
cls_embedding3

tensor([[-0.2015, -0.0688,  0.1207,  ..., -0.0610,  0.3574,  0.3081],
        [-0.0166, -0.0162,  0.1889,  ..., -0.1643,  0.4605, -0.0148],
        [-0.5549, -0.6477,  0.6902,  ..., -0.3045,  0.2095,  0.4269],
        ...,
        [ 0.0724,  0.4566,  0.0987,  ..., -0.1905,  0.2917, -0.5696],
        [-0.1629, -0.1394,  0.0578,  ...,  0.1936,  0.4371, -0.2020],
        [ 0.9252,  0.3082, -0.3010,  ...,  0.1568, -0.5969, -0.1635]])

In [ ]:
cls_embedding2

tensor([[ 9.2517e-01,  3.0822e-01, -3.0101e-01,  4.7908e-01, -3.9386e-01,
         -7.9996e-01,  4.0936e-01, -3.3544e-01,  6.0459e-01,  1.5788e-01,
          3.2878e-01, -1.7056e-01,  3.7715e-01, -5.5319e-02, -6.3176e-01,
         -7.8357e-02, -1.1143e-01, -5.8848e-02,  3.9124e-01,  8.3777e-02,
          6.2398e-01, -3.0552e-01,  6.3378e-01,  1.3704e-01,  1.8767e-01,
          2.9084e-01, -5.6048e-01,  4.5486e-02, -4.6161e-01, -5.0407e-01,
         -2.2868e-01, -4.1771e-01, -1.2273e-01,  4.2102e-01,  3.7814e-01,
         -1.3103e-01,  2.9615e-01,  6.9940e-02, -3.5369e-01, -4.5209e-01,
         -4.8102e-01,  2.9196e-01, -3.8154e-01,  2.9326e-01, -1.3298e-01,
         -4.7033e-01,  2.6733e-01,  3.1325e-01, -1.1071e-01,  6.1927e-01,
          1.7381e-01,  3.2380e-01, -2.0656e-01, -2.7290e-02,  3.0203e-01,
          2.3318e-01,  1.5106e-01, -7.1531e-01,  9.8843e-03, -2.1928e-02,
          1.2921e-01,  6.9806e-01, -4.1247e-01, -1.9097e-01,  4.2005e-01,
          2.2770e-02, -1.7920e-01, -1.

In [ ]:
cls_embedding1

tensor([[-1.6598e-02, -1.6250e-02,  1.8886e-01,  1.6606e-01,  1.3328e-01,
          1.2637e-01,  5.7070e-01,  5.2433e-01,  1.0405e-01, -9.1614e-02,
         -3.8632e-02, -4.3474e-01,  2.1243e-01,  5.1855e-01,  1.0874e-01,
          2.5485e-01,  1.4920e-01, -8.9226e-02, -8.2888e-02, -1.3754e-01,
         -2.8135e-01, -2.8285e-01, -2.0032e-01,  8.0535e-01,  2.1278e-01,
         -1.5092e-01,  4.2087e-02,  3.3375e-01,  1.7454e-01, -3.7740e-01,
          6.5581e-01,  3.4183e-01,  3.1558e-02, -2.8515e-01, -7.5867e-01,
          1.7018e-01, -4.4284e-01,  7.7506e-02,  1.3016e-01,  2.8330e-01,
         -3.9885e-01, -3.3802e-01,  3.7604e-01, -1.5366e-01, -3.3388e-01,
          9.2844e-02, -5.6121e-01, -1.6367e-01, -2.9149e-01, -3.8234e-01,
         -3.5035e-01, -1.2334e-01,  3.2070e-02,  2.6125e-01,  2.8671e-02,
          5.0789e-01, -5.1107e-01, -6.6422e-02,  2.4402e-01,  1.2804e-01,
          2.3853e-01,  2.9926e-01,  2.8493e-01, -3.1266e-01, -1.6440e-01,
         -4.8137e-01, -1.2374e-01,  1.

In [ ]:
cls_embedding


tensor([[-2.0151e-01, -6.8778e-02,  1.2073e-01, -1.7594e-01, -4.6503e-02,
         -2.4070e-01,  3.0335e-01,  5.4316e-01, -1.7485e-01, -1.5461e-01,
         -1.8231e-03, -1.2418e-01, -1.3184e-03,  3.5246e-01,  1.5064e-01,
          1.3530e-01, -1.2328e-01,  3.0548e-01,  1.6039e-01, -2.6012e-01,
         -1.3042e-01, -3.4376e-01,  5.7071e-02, -6.2676e-02,  6.1050e-02,
         -7.8781e-02, -7.6578e-02, -5.9111e-02,  2.7158e-01,  9.8975e-02,
          5.4113e-02,  1.2378e-01,  3.6470e-02, -1.7289e-01, -1.0377e-01,
          9.1288e-02,  8.7544e-02,  1.8524e-02,  8.8331e-02,  6.5841e-02,
         -1.0795e-01, -4.4374e-02,  1.5454e-01, -4.0554e-02, -2.1985e-01,
         -2.1346e-01, -2.3629e+00, -3.6666e-02, -1.0363e-01, -1.7586e-01,
          2.9385e-01,  1.8366e-02,  1.2149e-01,  1.7259e-01,  1.4576e-01,
          1.7673e-01, -4.7363e-01,  4.9239e-01, -3.7577e-02,  2.3654e-02,
          1.1391e-01,  1.4429e-01, -9.0516e-02,  1.5709e-03, -1.5315e-02,
         -3.2772e-02,  4.4378e-02,  2.

In [ ]:
cls_embedding.size()

torch.Size([1, 768])

#Distill Bert for tweet classiifcation

In [ ]:
df

,ID,tweet,labels,cleaned_tweet
0,1,@cath__kath AstraZeneca is made with the kidne...,ingredients,astrazeneca is made with the kidney cells of a...
1,2,It begins. Please find safe alternatives to th...,side-effect,it begins. please find safe alternatives to th...
2,3,"@PaolaQP1231 Well, I mean congratulations Covi...",side-effect,"well, i mean congratulations covid19 for being..."
3,4,@BorisJohnson for those of us that do not wish...,mandatory,for those of us that do not wish a vaccine so ...
4,5,She has been trying to speak out: writing lett...,side-effect rushed,she has been trying to speak out: writing lett...
...,...,...,...,...
9916,9917,Former Pfizer Chief Scientific Officer on Expe...,side-effect,former pfizer chief scientific officer on expe...
9917,9918,@garygilligan Not what the manufacturer's are ...,pharma,not what the manufacturer's are saying. the ma...
9918,9919,//That's a complete no for now on the Oxford/A...,none,//that's a complete no for now on the oxford/a...
9919,9920,Opinion: Vaccine side effects..- possible to h...,side-effect,opinion: vaccine side effects..- possible to h...


In [ ]:
list_of_tweets = df['cleaned_tweet'].to_list()
list_of_tweets

['astrazeneca is made with the kidney cells of a little girl aborted back in the 70s.',
 'it begins. please find safe alternatives to this vaccine. uk issues allergy warning about pfizer covid-19 vaccine after patients fall ill  via',
 'well, i mean congratulations covid19 for being the first ever “thing” to eradicate influenza. in other news, covid vaccines will spur the rise of influenza in  2021-2022 season. influenza will be returning for a shot at the title belt. nov 2, 2021 on pay per view. order today.',
 'for those of us that do not wish a vaccine so will not be given a vaccine passport to go abroad, how do you intend to deal with us, your objective i suppose is to push us to one side and penalise us, this is your great and fare justice system you talk about.',
 'she has been trying to speak out: writing letters to government, speaking to unison. she’s been questioning everything that’s been going on for a long time: “vaccine rollout, vaccine injury. realising it’s an experimen

In [ ]:
df.columns

Index(['ID', 'tweet', 'labels', 'cleaned_tweet'], dtype='object')

#Tokenization

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
token_lens = df['cleaned_tweet'].apply(lambda x: len(tokenizer.encode(x, truncation=True)))
max_length = token_lens.max()
print(max_length)
tokenized_data = tokenizer(list_of_tweets, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
tokenized_data

102


{'input_ids': tensor([[  101,  2004,  6494,  ...,     0,     0,     0],
        [  101,  2009,  4269,  ...,     0,     0,     0],
        [  101,  2092,  1010,  ...,     0,     0,     0],
        ...,
        [  101,  1013,  1013,  ...,     0,     0,     0],
        [  101,  5448,  1024,  ...,     0,     0,     0],
        [  101, 12897,  2085,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
tokenized_data['input_ids'].shape

torch.Size([9921, 102])

###checking whetehr gpu is available

In [ ]:
import torch
print(torch.cuda.is_available())

True


#Dataset and Dataloder

In [ ]:
from torch.utils.data import TensorDataset, random_split, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer

# Assuming you already have 'tokenized_data' and 'df' available

# Encode labels using MultiLabelBinarizer
labels_list = df['labels'].str.split().tolist()
label_encoder = MultiLabelBinarizer()
encoded_labels = label_encoder.fit_transform(labels_list)
labels_tensor = torch.tensor(encoded_labels)

# Create a TensorDataset
dataset = TensorDataset(tokenized_data['input_ids'], tokenized_data['attention_mask'], labels_tensor)

# Shuffle the entire dataset
indices = torch.randperm(len(dataset))

# Split the shuffled dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_indices = indices[:train_size]
val_indices = indices[train_size : train_size + val_size]
test_indices = indices[train_size + val_size :]

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# Create DataLoader instances for each set
batch_size = 16

# Training DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Validation DataLoader
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Testing DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
labels_tensor.shape

torch.Size([9921, 12])

In [ ]:
print(label_encoder.classes_)

['conspiracy' 'country' 'ineffective' 'ingredients' 'mandatory' 'none'
 'pharma' 'political' 'religious' 'rushed' 'side-effect' 'unnecessary']


#Model for classification

In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(distinct_labels_list))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##weights balance

In [ ]:
# Assuming ordered_classes is the correct order
#ordered_classes = ['conspiracy', 'country', 'ineffective', 'ingredients', 'mandatory', 'none','pharma', 'political', 'religious', 'rushed', 'side-effect', 'unnecessary']

# Frequencies of each class
ordered_classes = label_encoder.classes_
print(ordered_classes)
class_frequencies = {
    'conspiracy': 487,
    'country': 201,
    'ineffective': 1672,
    'ingredients': 436,
    'mandatory': 783,
    'none': 629,
    'pharma': 1273,
    'political': 626,
    'religious': 64,
    'rushed': 1477,
    'side-effect': 3805,
    'unnecessary': 722
}

# Calculate inverse class frequencies
total_samples = sum(class_frequencies.values())
class_weights = {cls: total_samples / freq for cls, freq in class_frequencies.items()}

# Normalize weights
total_weight = sum(class_weights.values())
normalized_class_weights = {cls: weight / total_weight for cls, weight in class_weights.items()}

# Display normalized weights
for cls, weight in normalized_class_weights.items():
    print(f"{cls}: {weight:.4f}")

# Convert weights to a list in the order of your classes
normalized_ordered_class_weights = [normalized_class_weights[cls] for cls in ordered_classes]
normalized_ordered_class_weights
# Now, you can use normalized_ordered_class_weights in your BCEWithLogitsLoss


['conspiracy' 'country' 'ineffective' 'ingredients' 'mandatory' 'none'
 'pharma' 'political' 'religious' 'rushed' 'side-effect' 'unnecessary']
conspiracy: 0.0620
country: 0.1502
ineffective: 0.0181
ingredients: 0.0693
mandatory: 0.0386
none: 0.0480
pharma: 0.0237
political: 0.0482
religious: 0.4718
rushed: 0.0204
side-effect: 0.0079
unnecessary: 0.0418


[0.061998371548266755,
 0.1502149599204274,
 0.01805813812440545,
 0.06925047464221538,
 0.038560928408692095,
 0.048001918829898106,
 0.023718151566383277,
 0.04823195997445034,
 0.4717688585000923,
 0.02044225250101957,
 0.007935139801315614,
 0.041818846182833665]

#Training

In [ ]:
from torch.nn import BCEWithLogitsLoss
from torch.optim import AdamW
from tqdm import tqdm

# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

class_weights_tensor = torch.tensor(normalized_ordered_class_weights, dtype=torch.float).to(device)

# Define the loss function suitable for multi-label classification with class weights
loss_function = BCEWithLogitsLoss(weight=class_weights_tensor).to(device)

# Define the loss function suitable for multi-label classification.
#loss_function = BCEWithLogitsLoss().to(device)

# Define the optimizer (AdamW is a variant of Adam optimized for transformer models)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Number of complete passes over the dataset
epochs = 3

# Start the training loop
for epoch in range(epochs):
    # Set the model to training mode. This activates dropout layers, batch normalization, etc.
    model.train()

    # Initialize variables to keep track of the cumulative loss and number of correct predictions for training
    total_loss = 0
    correct_predictions_train = 0
    total_predictions_train = 0

    # tqdm provides a progress bar for the loop
    for batch in tqdm(train_dataloader):

        # Zero the gradients. This is necessary as gradients accumulate in PyTorch
        model.zero_grad()

        # Move the batch tensors to the same device as the model
        input_ids, attention_mask, labels = tuple(b.to(device) for b in batch)

        # Forward pass: obtain logits from the model
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Compute the loss between the predicted logits and the true labels
        loss = loss_function(logits, labels.float())

        # Add the batch loss to the total loss
        total_loss += loss.item()

        # Backward pass: compute the gradient of the loss with respect to model parameters
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Convert logits to probabilities
        sigmoid_outputs = torch.sigmoid(logits)

        # Convert probabilities to binary predictions using a threshold of 0.5
        predictions_train = (sigmoid_outputs > 0.5).float()

        # Calculate the number of correct predictions in this batch for training
        correct_predictions_train += (predictions_train == labels.float()).sum().item()
        total_predictions_train += torch.numel(labels)

    # Calculate the average training loss
    avg_train_loss = total_loss / len(train_dataloader)

    # Calculate the training accuracy for this epoch
    accuracy_train = correct_predictions_train / total_predictions_train * 100

    # Print the average training loss and accuracy for this epoch
    print(f"Epoch {epoch + 1}, Train loss: {avg_train_loss}, Train Accuracy: {accuracy_train:.2f}%")

    # Validation loop
    model.eval()  # Set the model to evaluation mode

    # Initialize variables to keep track of the cumulative loss and number of correct predictions for validation
    total_loss_val = 0
    correct_predictions_val = 0
    total_predictions_val = 0

    with torch.no_grad():  # Disable gradient calculation during validation
        for batch_val in tqdm(val_dataloader):

            # Move the batch tensors to the same device as the model
            input_ids_val, attention_mask_val, labels_val = tuple(b.to(device) for b in batch_val)

            # Forward pass: obtain logits from the model
            outputs_val = model(input_ids_val, attention_mask=attention_mask_val)
            logits_val = outputs_val.logits

            # Compute the loss between the predicted logits and the true labels
            loss_val = loss_function(logits_val, labels_val.float())

            # Add the batch loss to the total loss for validation
            total_loss_val += loss_val.item()

            # Convert logits to probabilities
            sigmoid_outputs_val = torch.sigmoid(logits_val)

            # Convert probabilities to binary predictions using a threshold of 0.5
            predictions_val = (sigmoid_outputs_val > 0.5).float()

            # Calculate the number of correct predictions in this batch for validation
            correct_predictions_val += (predictions_val == labels_val.float()).sum().item()
            total_predictions_val += torch.numel(labels_val)

    # Calculate the average validation loss
    avg_val_loss = total_loss_val / len(val_dataloader)

    # Calculate the validation accuracy for this epoch
    accuracy_val = correct_predictions_val / total_predictions_val * 100

    # Print the average validation loss and accuracy for this epoch
    print(f"Epoch {epoch + 1}, Validation loss: {avg_val_loss}, Validation Accuracy: {accuracy_val:.2f}%")


100%|██████████| 434/434 [01:01<00:00,  7.06it/s]


Epoch 1, Train loss: 0.011521143963225702, Train Accuracy: 89.34%


100%|██████████| 93/93 [00:03<00:00, 24.73it/s]


Epoch 1, Validation loss: 0.008330364501284014, Validation Accuracy: 91.44%


100%|██████████| 434/434 [01:00<00:00,  7.17it/s]


...

Epoch 2, Validation loss: 0.006694251511205909, Validation Accuracy: 93.11%


100%|██████████| 434/434 [01:02<00:00,  6.96it/s]


Epoch 3, Train loss: 0.0049156314963775295, Train Accuracy: 94.36%


100%|██████████| 93/93 [00:04<00:00, 22.48it/s]

Epoch 3, Validation loss: 0.006930269571321626, Validation Accuracy: 94.10%


#Testing

In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize a variable to keep track of the number of correct predictions
test_accuracy = 0
TP = torch.zeros(12).to(device)
TN = torch.zeros(12).to(device)
FP = torch.zeros(12).to(device)
FN = torch.zeros(12).to(device)

# Loop through batches in the test dataloader
for batch in test_dataloader:
    # Move batch tensors to the device the model is on
    input_ids, attention_mask, labels = tuple(b.to(device) for b in batch)

    # Make predictions with the model
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Extract the logits from the model outputs
    logits = outputs.logits

    # Convert logits to probabilities
    sigmoid_outputs = torch.sigmoid(logits)

    # Convert probabilities to binary predictions using a threshold of 0.5
    predictions = (sigmoid_outputs > 0.5).float()

    TP += (predictions * labels).sum(dim=0)
    TN += ((1 - predictions) * (1 - labels)).sum(dim=0)
    FP += (predictions * (1 - labels)).sum(dim=0)
    FN += ((1 - predictions) * labels).sum(dim=0)

    # Calculate the number of correct predictions in this batch
    correct_predictions = (predictions == labels).float().sum().item()

    # Add the number of correct predictions from this batch to the total
    test_accuracy += correct_predictions


accuracy_percentage = (test_accuracy / (test_size * 12)) * 100  # since we have 12 labels per example
recall = TP / (TP + FN)
specificity = TN / (TN + FP)
precision = TP / (TP + FP)
f1_score = 2 * (precision * recall) / (precision + recall)


# Print the overall accuracy
print("Test Accuracy:", accuracy_percentage, "%")

#print metrics
for i in range(12):
    print(f"{label_encoder.classes_[i]}: Recall: {recall[i]:.4f}, Specificity: {specificity[i]:.4f}, Precision: {precision[i]:.4f}, F1-Score: {f1_score[i]:.4f}")



Test Accuracy: 93.7038280725319 %
conspiracy: Recall: 0.4186, Specificity: 0.9922, Precision: 0.7660, F1-Score: 0.5414
country: Recall: 0.6538, Specificity: 0.9952, Precision: 0.7083, F1-Score: 0.6800
ineffective: Recall: 0.5160, Specificity: 0.9694, Precision: 0.7967, F1-Score: 0.6263
ingredients: Recall: 0.6667, Specificity: 0.9797, Precision: 0.5915, F1-Score: 0.6269
mandatory: Recall: 0.5039, Specificity: 0.9934, Precision: 0.8784, F1-Score: 0.6404
none: Recall: 0.2892, Specificity: 0.9872, Precision: 0.5714, F1-Score: 0.3840
pharma: Recall: 0.5241, Specificity: 0.9570, Precision: 0.6364, F1-Score: 0.5748
political: Recall: 0.1905, Specificity: 0.9964, Precision: 0.7619, F1-Score: 0.3048
religious: Recall: 0.3333, Specificity: 0.9973, Precision: 0.3333, F1-Score: 0.3333
rushed: Recall: 0.5862, Specificity: 0.9603, Precision: 0.7000, F1-Score: 0.6381
side-effect: Recall: 0.7653, Specificity: 0.9059, Precision: 0.8281, F1-Score: 0.7955
unnecessary: Recall: 0.3148, Specificity: 0.9841

In [ ]:
print(logits)


tensor([[-6.4056, -8.7808, -4.2773, -5.9169, -6.0629, -6.6754, -5.4638, -6.5587,
         -9.2453, -5.3820,  3.5361, -6.1628]], device='cuda:0')


In [ ]:
type(label_encoder.classes_)

numpy.ndarray

In [ ]:
logits.shape

torch.Size([1, 12])

In [ ]:
batch[2].shape

torch.Size([1, 12])

In [ ]:
sigmoid_outputs

tensor([[1.6496e-03, 1.5363e-04, 1.3690e-02, 2.6862e-03, 2.3222e-03, 1.2600e-03,
         4.2196e-03, 1.4157e-03, 9.6555e-05, 4.5776e-03, 9.7170e-01, 2.1019e-03]],
       device='cuda:0')

In [ ]:
sigmoid_outputs.shape

torch.Size([1, 12])

In [ ]:
labels.shape

torch.Size([1, 12])

In [ ]:
predictions.shape

torch.Size([1, 12])

#Inference

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/My_model")


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/My_model"

config.json  model.safetensors	pytorch_model.bin


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer_path = "/content/drive/MyDrive/Colab Notebooks/Tokenizer"
tokenizer.save_pretrained(tokenizer_path)

('/content/drive/MyDrive/Colab Notebooks/Tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Tokenizer/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Tokenizer/added_tokens.json')

In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/My_model"

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

In [ ]:
# Tokenize input text
input_text = "Staying resilient in the face of COVID-19. Grateful for unity and kindness. We'll get through this together."
tokens = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

# Run inference
outputs = model(**tokens)

# Get predicted class probabilities
probabilities = outputs.logits.softmax(dim=1)


In [ ]:
probabilities

tensor([[6.6793e-03, 6.2817e-04, 1.5097e-01, 1.4998e-03, 3.0174e-02, 2.2146e-01,
         2.9526e-02, 7.2400e-03, 7.6988e-05, 1.0054e-02, 7.2359e-02, 4.6933e-01]],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
threshold = 0.5
predicted_labels_tensor = (probabilities > threshold).float()
predicted_labels_numpy = predicted_labels_tensor.numpy()
predicted_labels_list = label_encoder.inverse_transform(predicted_labels_numpy)


In [ ]:
predicted_labels_tensor

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
predicted_labels_list

[()]

In [ ]:
predicted_labels_tensor

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
predicted_labels_numpy

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
label_encoder.classes_

array(['conspiracy', 'country', 'ineffective', 'ingredients', 'mandatory',
       'none', 'pharma', 'political', 'religious', 'rushed',
       'side-effect', 'unnecessary'], dtype=object)

In [ ]:
predicted_labels_list

[()]